# SQL Server 2019 Data Virtualization - Using Polybase to query SAP HANA
This notebook contains an example of how to use external tables to query data in SAP HANA without moving data. You may need to change identity, secret, connection, database, schema, and remote table names to work with your Oracle Database.

This notebook also assumes you are using SQL Server 2019 Release Candidate or later and that the Polybase feature has been installed and enabled.

This notebook uses the sample WideWorldImporters sample database but can be used with any user database.

## Step 0: Create a database in SAP HANA, table, and data

This example uses a SAP HANA Express in Azure. You can use a tool like hdbsql to execute SQL statements to create the database, user, table, and add data.

Create the database using the following SQL statement:

```sql
CREATE DATABASE VANDELAY SYSTEM USER PASSWORD <password>;
```

Create a user using the following SQL statements connected as SYSTEM with the context of the database you created.

```sql
CREATE USER bwsaphana PASSWORD <password> NO FORCE_FIRST_PASSWORD_CHANGE;
GRANT AFLPM_CREATOR_ERASER_EXECUTE TO bwsaphana;
GRANT CONTENT_ADMIN TO bwsaphana;
GRANT MODELING to bwsaphana;
```
Create a table using the following SQL statement connected as the USER

```sql
CREATE TABLE Customers (CustomerID int, CustomerName nvarchar(100), AccountOpenedDate date, CustomerWebSite nvarchar(256), primary key (CustomerID));
```

Insert data into the table using the following SQL statement connected as the USER:

```sql
INSERT INTO Customers VALUES (100000, 'Costanza Architecture', '2000-01-01', 'www.costanza.com');
```

## Step 1: Create a master key
Create a master key to encrypt the database credential

In [7]:
USE [WideWorldImporters]
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<password>'
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

## Step 2: Create a database credential
The database credential contains the IDENTITY (login) and SECRET (password) of SAP HANA database. Change this to the login and password created in Step 0.

In [8]:
CREATE DATABASE SCOPED CREDENTIAL SAPHANACredentials   
WITH IDENTITY = 'bwsaphana', Secret = '<password>'
GO

Commands completed successfully.

Total execution time: 00:00:00.005

## Step 3: Create an EXTERNAL DATA SOURCE
The EXTERNAL DATA SOURCE indicates what type of data source, the connection "string", where PUSHDOWN predicates should be used (if possible), and the name of the database credential.

The LOCATION syntax is <datasourcetype>:<connection string>.

datasourcetype can be sqlserver, oracle, teradata, mongodb, or odbc (Windows only)
The connection string depends on the datasourcetype

Since SAP HANA is through an ODBC driver not isntalled by default, the LOCATION is the name of the host for the SAP HANA server. Use the CONNECTION_OPTIONS for the ODBC data source connecting string including the name of the driver, the name of the server, and the SAP HANA port.

In [9]:
CREATE EXTERNAL DATA SOURCE SAPHANAServer
WITH ( 
LOCATION = 'odbc://<SAP HANA Host>',
CONNECTION_OPTIONS = 'Driver={HDBODBC};ServerNode=<SAP HANA Host>:<port>',
PUSHDOWN = ON,
CREDENTIAL = SAPHANACredentials
)
GO

Commands completed successfully.

Total execution time: 00:00:00.021

## Step 4: Create a schema for the EXTERNAL TABLE
Schemas provide convenient methods to secure and organize objects

In [10]:
CREATE SCHEMA saphana
GO

Commands completed successfully.

Total execution time: 00:00:00.007

## Step 5: Create an EXTERNAL TABLE
An external table provides metadata so SQL Server knows how to map columns to the remote table. The name of the tables for the external table can be your choice. But the columns must be specified in the same order with the same name as they are defined in the remote table. Furthermore, local data types must be compatible with the remote table.

The WITH clause specifies a LOCATION. This LOCATION is different than the EXTERNAL DATA SOURCE. For SAP HANA, this LOCATION indicates the [schema].[table] of the SAP HANA table. The schema is the USER you created in Step 0. The DATA_SOURCE clause is the name of the EXTERNAL DATA SOURCE you created earlier. You don't need the name of the database since the database scoped credentials are for a USER that is default for the database you created in Step 0.

In [12]:
CREATE EXTERNAL TABLE saphana.customers
(
CUSTOMERID int,
CUSTOMERNAME nvarchar(100) COLLATE Latin1_General_100_CI_AS,
ACCOUNTOPENEDDATE date,
CUSTOMERWEBSITE nvarchar(256) COLLATE Latin1_General_100_CI_AS
)
 WITH (
 LOCATION='[BWSAPHANA].[CUSTOMERS]',
 DATA_SOURCE=SAPHANAServer
)
GO

Commands completed successfully.

Total execution time: 00:00:01.055

## Step 6: Create statistics
SQL Server allows you to store local statistics about specific columns from the remote table. This can help the query processing to make more efficient plan decisions.

In [13]:
CREATE STATISTICS customerstats ON saphana.customers ([CustomerName]) WITH FULLSCAN
GO

Commands completed successfully.

Total execution time: 00:00:00.630

## Step 7: Try to scan the remote table
Run a simple query on the EXTERNAL TABLE to scan all rows.

In [14]:
SELECT * FROM saphana.customers
GO

(1 row affected)

Total execution time: 00:00:00.263

CUSTOMERID,CUSTOMERNAME,ACCOUNTOPENEDDATE,CUSTOMERWEBSITE
100000,Costanza Architecture,2000-01-01,www.costanza.com


## Step 8: Get all customers
Use a UNION to get customers from SAP HANA and local SQL Server.


In [15]:
SELECT CustomerID, CustomerName, AccountOpenedDate, CustomerWebSite
FROM saphana.customers
UNION
SELECT CustomerID, CustomerName, AccountOpenedDate, WebsiteURL
FROM Sales.Customers
GO

(664 rows affected)

Total execution time: 00:00:00.501

CustomerID,CustomerName,AccountOpenedDate,CustomerWebSite
1,Tailspin Toys (Head Office),2013-01-01,http://www.tailspintoys.com
2,"Tailspin Toys (Sylvanite, MT)",2013-01-01,http://www.tailspintoys.com/Sylvanite
3,"Tailspin Toys (Peeples Valley, AZ)",2013-01-01,http://www.tailspintoys.com/PeeplesValley
4,"Tailspin Toys (Medicine Lodge, KS)",2013-01-01,http://www.tailspintoys.com/MedicineLodge
5,"Tailspin Toys (Gasport, NY)",2013-01-01,http://www.tailspintoys.com/Gasport
6,"Tailspin Toys (Jessie, ND)",2013-01-01,http://www.tailspintoys.com/Jessie
7,"Tailspin Toys (Frankewing, TN)",2013-01-01,http://www.tailspintoys.com/Frankewing
8,"Tailspin Toys (Bow Mar, CO)",2013-01-01,http://www.tailspintoys.com/BowMar
9,"Tailspin Toys (Netcong, NJ)",2013-01-01,http://www.tailspintoys.com/Netcong
10,"Tailspin Toys (Wimbledon, ND)",2013-01-01,http://www.tailspintoys.com/Wimbledon
